In [ ]:
from urllib.request import urlopen
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB,BernoulliNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np # linear algebra
import pandas as pd 
pd.options.mode.chained_assignment = None
import datetime as DT
import io
import requests

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive"

Mounted at /content/drive
 1a_solution_to_score_8_0_in_ielts_writing.pdf	 grouped_monthly_counties.csv
'Colab Notebooks'				'Linear Models'
'Copia di MDA'					 MDA-2022-Italy
 CV_Casella.pdf					'Motivation Letter iGEM.odt'
 DataMining					 Report.R
'ENERGY AND BUILDINGS - DECEMBER.gsheet'


In [ ]:
covid_policy = pd.read_excel("/content/drive/MyDrive/MDA-2022-Italy/Report/Dataset_preparation/states_covid_policy_internal.xlsx")
covid_policy

,state_code,date,inmate_vaccination,inmate_testing,inmate_masking,inmate_releases,inmate_social_distance,inmate_social_distance_sleep,inmate_screening,inmate_quarantine,inmate_visitation_suspended,staff_testing,staff_masking,staff_quarantine
0,AK,2020-04-01,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,AK,2020-04-02,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,AK,2020-04-03,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,AK,2020-04-04,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,AK,2020-04-05,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18103,WY,2021-03-27,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0
18104,WY,2021-03-28,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0
18105,WY,2021-03-29,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0
18106,WY,2021-03-30,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0


In [ ]:
covid_policy.dtypes

state_code                       object
date                             object
inmate_vaccination              float64
inmate_testing                  float64
inmate_masking                  float64
inmate_releases                 float64
inmate_social_distance          float64
inmate_social_distance_sleep    float64
inmate_screening                float64
inmate_quarantine               float64
inmate_visitation_suspended     float64
staff_testing                   float64
staff_masking                   float64
staff_quarantine                float64
dtype: object

In [ ]:
covid_policy_clean = covid_policy.fillna(0)
covid_policy_clean.isna().sum()

state_code                      0
date                            0
inmate_vaccination              0
inmate_testing                  0
inmate_masking                  0
inmate_releases                 0
inmate_social_distance          0
inmate_social_distance_sleep    0
inmate_screening                0
inmate_quarantine               0
inmate_visitation_suspended     0
staff_testing                   0
staff_masking                   0
staff_quarantine                0
dtype: int64

In [ ]:
# Remove rows that have completely missing values
#covid_policy_clean = covid_policy.dropna(axis=0)
# covid_policy_clean

In [ ]:
# Compare the dimensions of original and cleaned data 
print('dimensions of the original dataset:', covid_policy.shape)
print('dimensions after removing NaN rows:', covid_policy_clean.shape)

dimensions of the original dataset: (18108, 14)
dimensions after removing NaN rows: (18108, 14)


In [ ]:
covid_policy_clean = covid_policy_clean.astype({col: int for col in covid_policy_clean.columns[2:]})
covid_policy_clean["date"] = pd.to_datetime(covid_policy_clean["date"])
covid_policy_clean.dtypes

state_code                              object
date                            datetime64[ns]
inmate_vaccination                       int64
inmate_testing                           int64
inmate_masking                           int64
inmate_releases                          int64
inmate_social_distance                   int64
inmate_social_distance_sleep             int64
inmate_screening                         int64
inmate_quarantine                        int64
inmate_visitation_suspended              int64
staff_testing                            int64
staff_masking                            int64
staff_quarantine                         int64
dtype: object

In [ ]:
covid_policy_clean.head(100)

# Create a column that converts date to its respective quarter
covid_policy_clean['quarter'] = covid_policy_clean['date'].dt.to_period("Q")
covid_policy_clean.head(100)
# Place new quarter column after date column
col = covid_policy_clean.pop("quarter")
covid_policy_clean.insert(2, "quarter", col)
covid_policy_clean.head(100)


,state_code,date,quarter,inmate_vaccination,inmate_testing,inmate_masking,inmate_releases,inmate_social_distance,inmate_social_distance_sleep,inmate_screening,inmate_quarantine,inmate_visitation_suspended,staff_testing,staff_masking,staff_quarantine
0,AK,2020-04-01,2020Q2,0,0,0,0,0,0,1,0,1,0,0,0
1,AK,2020-04-02,2020Q2,0,0,0,0,0,0,1,0,1,0,0,0
2,AK,2020-04-03,2020Q2,0,0,0,0,0,0,1,0,1,0,0,0
3,AK,2020-04-04,2020Q2,0,0,0,0,0,0,1,0,1,0,0,0
4,AK,2020-04-05,2020Q2,0,0,0,0,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,AK,2020-07-05,2020Q3,0,0,0,0,0,1,1,0,1,0,0,0
96,AK,2020-07-06,2020Q3,0,0,0,0,0,1,1,0,1,0,0,0
97,AK,2020-07-07,2020Q3,0,0,0,0,0,1,1,0,1,0,0,0
98,AK,2020-07-08,2020Q3,0,0,0,0,0,1,1,0,1,0,0,0


In [ ]:
# Creates duplicate dataframe to calculate and assign weights for each variable
policy_score = covid_policy_clean

In [ ]:
#Covid_policy2=Covid_policy1.groupby([pd.Grouper(key='date', freq='QS'), 'STATE']).agg({'Inmate_Vaccination':'median', 'Inmate_Testing':'median','Inmate_Masking':'median', 'Inmate_Releases':'median','Inmate_SocialDistance':'median', 'Inmate_Quarantine':'median','Inmate_Visitation':'median'})
#Covid_policy2


covid_policy_test = covid_policy_clean.groupby(['state_code', 'quarter']).agg({'inmate_vaccination':'median', 
                                                               'inmate_testing':'median',
                                                               'inmate_masking':'median', 
                                                               'inmate_releases':'median',
                                                               'inmate_social_distance':'median',
                                                               'inmate_social_distance_sleep':'median',
                                                               'inmate_screening':'median',
                                                               'inmate_quarantine':'median', 
                                                               'inmate_visitation_suspended':'median',
                                                               'staff_testing':'median',
                                                               'staff_masking':'median',
                                                               'staff_quarantine':'median'})
covid_policy_test

inmate_vaccination  inmate_testing  inmate_masking  \
state_code quarter                                                       
AK         2020Q2                  0.0             0.0             0.0   
           2020Q3                  0.0             0.0             0.0   
           2020Q4                  0.0             1.0             0.0   
           2021Q1                  1.0             1.0             0.0   
AL         2020Q2                  0.0             0.0             0.0   
...                                ...             ...             ...   
WV         2021Q1                  0.0             1.0             1.0   
WY         2020Q2                  0.0             0.0             0.0   
           2020Q3                  0.0             1.0             0.0   
           2020Q4                  0.0             0.0             0.0   
           2021Q1                  0.0             0.0             0.0   

                    inmate_releases  inmate_social_distance  \
state_code quarter                                            
AK         2020Q2               0.0                     0.0   
           2020Q3               0.0                     0.0   
           2020Q4               0.0                     0.0   
           2021Q1               0.0                     0.0   
AL         2020Q2               0.0                     0.0   
...                             ...                     ...   
WV         2021Q1               1.0                     0.0   
WY         2020Q2               0.0                     1.0   
           2020Q3               0.0                     1.0   
           2020Q4               0.0                     1.0   
           2021Q1               0.0                     1.0   

                    inmate_social_distance_sleep  inmate_screening  \
state_code quarter                                                   
AK         2020Q2                            1.0               1.0   
           2020Q3                            1.0               1.0   
           2020Q4                            1.0               1.0   
           2021Q1                            1.0               1.0   
AL         2020Q2                            0.0               1.0   
...                                          ...               ...   
WV         2021Q1                            0.0               1.0   
WY         2020Q2                            0.0               0.0   
           2020Q3                            0.0               1.0   
           2020Q4                            0.0               1.0   
           2021Q1                            0.0               1.0   

                    inmate_quarantine  inmate_visitation_suspended  \
state_code quarter                                                   
AK         2020Q2                 0.0                          1.0   
           2020Q3                 0.0                          1.0   
           2020Q4                 0.0                          1.0   
           2021Q1                 0.0                          1.0   
AL         2020Q2                 1.0                          1.0   
...                               ...                          ...   
WV         2021Q1                 1.0                          1.0   
WY         2020Q2                 0.0                          1.0   
           2020Q3                 0.0                          1.0   
           2020Q4                 0.0                          1.0   
           2021Q1                 0.0                          1.0   

                    staff_testing  staff_masking  staff_quarantine  
state_code quarter                                                  
AK         2020Q2             0.0            0.0               0.0  
           2020Q3             0.0            1.0               0.0  
           2020Q4             0.0            1.0               0.0  
           2021Q1             0.0            1.0               0.0  
A

In [ ]:
covid_policy_test.isna().sum()

inmate_vaccination              0
inmate_testing                  0
inmate_masking                  0
inmate_releases                 0
inmate_social_distance          0
inmate_social_distance_sleep    0
inmate_screening                0
inmate_quarantine               0
inmate_visitation_suspended     0
staff_testing                   0
staff_masking                   0
staff_quarantine                0
dtype: int64

In [ ]:
# Reset index to "undo" grouping to allow dataframe manipulations
covid_policy_test = covid_policy_test.reset_index()
covid_policy_test

,state_code,quarter,inmate_vaccination,inmate_testing,inmate_masking,inmate_releases,inmate_social_distance,inmate_social_distance_sleep,inmate_screening,inmate_quarantine,inmate_visitation_suspended,staff_testing,staff_masking,staff_quarantine
0,AK,2020Q2,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
1,AK,2020Q3,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
2,AK,2020Q4,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
3,AK,2021Q1,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
4,AL,2020Q2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,WV,2021Q1,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
195,WY,2020Q2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
196,WY,2020Q3,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0
197,WY,2020Q4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0


In [ ]:
covid_policy_test.isna().sum()

state_code                      0
quarter                         0
inmate_vaccination              0
inmate_testing                  0
inmate_masking                  0
inmate_releases                 0
inmate_social_distance          0
inmate_social_distance_sleep    0
inmate_screening                0
inmate_quarantine               0
inmate_visitation_suspended     0
staff_testing                   0
staff_masking                   0
staff_quarantine                0
dtype: int64

In [ ]:
 d1 = covid_policy_test.set_index(['state_code', 'quarter']).unstack()
d1.columns = d1.columns.map(lambda x: '{}_quarter{}'.format(*x))
d1.reset_index(inplace=True)
d1 = d1.fillna(0) # To account for Tenessee missing any data for 2021

d1

,state_code,inmate_vaccination_quarter2020Q2,inmate_vaccination_quarter2020Q3,inmate_vaccination_quarter2020Q4,inmate_vaccination_quarter2021Q1,inmate_testing_quarter2020Q2,inmate_testing_quarter2020Q3,inmate_testing_quarter2020Q4,inmate_testing_quarter2021Q1,inmate_masking_quarter2020Q2,...,staff_testing_quarter2020Q4,staff_testing_quarter2021Q1,staff_masking_quarter2020Q2,staff_masking_quarter2020Q3,staff_masking_quarter2020Q4,staff_masking_quarter2021Q1,staff_quarantine_quarter2020Q2,staff_quarantine_quarter2020Q3,staff_quarantine_quarter2020Q4,staff_quarantine_quarter2021Q1
0,AK,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,AL,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,AR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
3,AZ,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
4,CA,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,CO,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
6,CT,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,DE,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
8,FL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9,GA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0


In [ ]:
d1.isna().sum()

state_code                                    0
inmate_vaccination_quarter2020Q2              0
inmate_vaccination_quarter2020Q3              0
inmate_vaccination_quarter2020Q4              0
inmate_vaccination_quarter2021Q1              0
inmate_testing_quarter2020Q2                  0
inmate_testing_quarter2020Q3                  0
inmate_testing_quarter2020Q4                  0
inmate_testing_quarter2021Q1                  0
inmate_masking_quarter2020Q2                  0
inmate_masking_quarter2020Q3                  0
inmate_masking_quarter2020Q4                  0
inmate_masking_quarter2021Q1                  0
inmate_releases_quarter2020Q2                 0
inmate_releases_quarter2020Q3                 0
inmate_releases_quarter2020Q4                 0
inmate_releases_quarter2021Q1                 0
inmate_social_distance_quarter2020Q2          0
inmate_social_distance_quarter2020Q3          0
inmate_social_distance_quarter2020Q4          0
inmate_social_distance_quarter2021Q1    

In [ ]:
from google.colab import files

d1.to_csv('covid_policydata.csv', encoding = 'utf-8-sig') 
files.download('covid_policydata.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# covid_policy_test2 = covid_policy_test.pivot(index='state_code', columns='quarter', 
  #                                           values=['inmate_vaccination', 
   #                                                  'inmate_testing',                        
    #                                                 'inmate_masking',
     #                                                'inmate_releases',                       
       #                                              'inmate_social_distance',              
        #                                             'inmate_social_distance_sleep',  
         #                                            'inmate_screening',                     
          #                                           'inmate_quarantine',                     
           #                                          'inmate_visitation_suspended',           
            #                                         'staff_testing',                       
             #                                        'staff_masking',                        
              #                                       'staff_quarantine'])

# covid_policy_test2

In [ ]:
# SECTION FOR CREATING VARIABLE WEIGHTS BY QUARTER


# Creates duplicate dataframe to calculate and assign weights for each variable
policy_score = covid_policy_clean

# Drops duplicated states and date combinations and groups by state and quarter to determine sum scores 

policy_score_clean = policy_score.drop_duplicates(subset=['state_code', 'date']).groupby(["state_code", "quarter"]).sum().reset_index()
#policy_score_clean.describe(include='int64') # Max score is 92 (Consistent with amount of days in corresponding quarter)
# policy_score_clean.head(20)

# Dividing select columns by 7 (days per week) and rounding to nearest integer
cols = ['inmate_vaccination', 'inmate_testing', 'inmate_masking', 'inmate_releases',
        'inmate_social_distance', 'inmate_social_distance_sleep','inmate_screening',
        'inmate_quarantine', 'inmate_visitation_suspended', 'staff_testing',
        'staff_masking', 'staff_quarantine']

policy_score_clean[cols] = policy_score_clean[cols].div(7).round()

# Converts to integers for simplification
policy_score_clean = policy_score_clean.astype({col: int for col in policy_score_clean.columns[2:]})

#policy_score_clean.describe()
#policy_score_clean.head(20)

# Transposes dataframe by policy variable
policy_score_clean = policy_score_clean.melt(id_vars=["state_code", "quarter"], 
        var_name="policy", 
        value_name="score")

# Convert var quarter to string to clean variable (remove year)
policy_score_clean['quarter'] = policy_score_clean['quarter'].astype(str)

# Removes the year from the quarter column (to help with filtering)
policy_score_clean['quarter'] = policy_score_clean['quarter'].str[4:]
# policy_score_clean.head(20)

# Drop row if score = 0 (To avoid defining neo4j relationships with 0 weight)
policy_score_clean = policy_score_clean.drop(policy_score_clean[policy_score_clean.score == 0].index)

# Exports file to excel
from google.colab import files

policy_score_clean.to_csv('policy_scores.csv', encoding = 'utf-8-sig') 
files.download('policy_scores.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Create an excel sheet for policy by quarter
policy_scores_q1 = policy_score_clean[policy_score_clean["quarter"] == "Q1"] 

In [ ]:
policy_scores_q1

,state_code,quarter,policy,score
3,AK,Q1,inmate_vaccination,13
19,CA,Q1,inmate_vaccination,10
23,CO,Q1,inmate_vaccination,12
27,CT,Q1,inmate_vaccination,8
51,ID,Q1,inmate_vaccination,5
...,...,...,...,...
2284,MO,Q1,staff_quarantine,2
2304,NE,Q1,staff_quarantine,2
2312,NJ,Q1,staff_quarantine,13
2316,NM,Q1,staff_quarantine,13
